# Importing the Required Library

In [ ]:
from google.colab import drive
import pandas as pd
import numpy as np
from google.cloud import bigquery
from datetime import date
drive.mount('/content/drive') #mounting g drive

## Visualization Library

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns

### Parameter for visualization
The following settings will improve the default style and font sizes for our charts.

In [ ]:
sns.set_style('darkgrid') #theme of graph
matplotlib.rcParams['font.size'] = 14 #fontsize displayed in graph
matplotlib.rcParams['figure.figsize'] = (10, 6) #Chart size parameter
matplotlib.rcParams['figure.facecolor'] = '#00000000'

In [ ]:
#Setting up conection between notebook and Database

folder_path = '/content/drive/MyDrive/pran-project-470608-2f7bfba6b03c.json'
client = bigquery.Client.from_service_account_json(folder_path)

### Loading Data from Database using the connection we setup

In [ ]:
query1 = """
select * from firmable.data_table
"""
df = client.query(query1).to_dataframe()
print("size of dataframe",df.shape)

In [ ]:
query2 = "select * from firmable.company_details"
df2 = client.query(query2).to_dataframe()
print("size of dataframe",df2.shape)

In [ ]:
query3 = "select * from firmable.article_details"
df3 = client.query(query3).to_dataframe()
print("size of dataframe",df3.shape)

df1 620785 and df3 has 577277  rows while df2 has 148630 rows as duplicate rows were in ingestion code

Merging all dataframe so that we can work on single dataframe

In [ ]:
merge_df = df.merge(df2,left_on = 'relationship_company1_id',right_on = 'id',how = 'left',suffixes = ('','_company_1'))

final_merge = merge_df.merge(df2,left_on = 'relationship_company2_id',right_on = 'id',how = 'left',suffixes = ('','_company_2'))

final_df = final_merge.merge(df3,left_on ='relationship_most_relevant_id',right_on='id',how= 'left',suffixes = ('','_article') )
print("size of dataframe",final_df.shape)

after merging of all three rows we are getting 620785 rows
As every article id should be unique to every event as article_table< data_table which indicate presnt of duplicate article_id in dataset

In [ ]:
final_df.head(5)

Checking Missing value

In [ ]:
null_column_count = pd.DataFrame({
    'null_value': final_df.isnull().sum(),
    'total_rows':len(final_df),
    'null_percentage': (final_df.isnull().sum() / len(final_df)* 100).round(2)
})

null_column_count= null_column_count.sort_values(by = ['null_percentage','null_value'],ascending = [False,False])
null_column_count

After analyisng the dataframe above
we can say column division has all the value null
there are many other column as well like product_version,financing_type_normalized,headcount,assets,award,event,product_release_type with more than 95% values are null

We can't drop this column or fill the value in this as all the rows are mutually exlusive to each other

In [ ]:
final_df.dtypes

DataTypes Check
Effective date column is in dbDate it only represnt date part only
and we can't directly compare column with datetime column

product_fuzzy_match value has few null column else it contain Boolean valuer that alos need to be corrected

In [ ]:
# temp_df = final_df.copy()
# temp_df["effective_date"] = pd.to_datetime(temp_df["effective_date"], errors="coerce", utc=True)
# temp_df.dtypes

The describe() function in pandas provides the statistical distribution of all numeric columns in the dataset.

From the output, we observe the following

1. Headcount is currently stored as a float datatype. Since headcount represents the number of individuals, it should be stored as an integer. Additionally, the minimum value of headcount is negative, which is not logically possible and should be treated as a data quality issue.


2. Amount_normalized also contains negative values, which are invalid in the given business context and should be corrected or removed during data cleaning.

3. Confidence_score is stored as a float and is correctly constrained within the range [0, 1], which aligns with its definition as a probability or model score.

In [ ]:
final_df.describe()

# Distribution of categry

Here we will be checking Distribution of major column values

In [ ]:
final_df.category.value_counts(ascending = False)

In [ ]:
plt.bar(final_df['category'].unique(), final_df['category'].value_counts())

plt.title("Category Distribution")
plt.xlabel("Category")
plt.ylabel("Count")
plt.xticks(final_df['category'].unique(),rotation = 90)
plt.tight_layout()

plt.show()

Distribution of news over the year

In [ ]:
 year_count = final_df.copy()
# Ensure found_date is datetime
year_count["found_date"] = pd.to_datetime(year_count["found_date"], errors="coerce")

# Extract year-month
year_count["year"] = year_count["found_date"].dt.year

# Count how many news ingested in each year-month
news_counts = year_count.groupby("year").size().reset_index(name = "news_count").sort_values("year")

news_counts

In [ ]:
plt.bar(news_counts['year'], news_counts['news_count'])

plt.title("Number of News Ingested per Year")
plt.xlabel("Year")
plt.ylabel("News Count")
plt.xticks(news_counts['year'], rotation=45)  # show year on X-axis
plt.tight_layout()

# Add labels to the bars
for index, row in news_counts.iterrows():
    plt.text(row['year'], row['news_count'], str(row['news_count']), ha='center', va='bottom', rotation=90)

plt.show()

Count of Article id to check repetation as it should be unique

In [ ]:
final_df.id_article.value_counts(ascending = False)

**When we checked distribution of article id we found out ther are duplicate article id present
which Directly hints towards duplicacy of rows in dataframe**

## Accuracy

Check wether article published correctly represnt actual news and facts

Start with date as news which is ingested into system should happen or published the ingestion date

In [ ]:
date_accuracy_check = final_df.copy()
date_accuracy_check['found_date'] = pd.to_datetime(date_accuracy_check['found_date'],utc= True)
date_accuracy_check['effective_date']= pd.to_datetime(date_accuracy_check['effective_date'],utc= True)
date_accuracy_check['published_at']= pd.to_datetime(date_accuracy_check['published_at'],utc= True)

date_accuracy_check['reference_date'] = date_accuracy_check['effective_date'].fillna(
    date_accuracy_check['published_at']
)

date_inconsitency_count = date_accuracy_check[
    (date_accuracy_check['reference_date'].dt.date > date_accuracy_check['found_date'].dt.date) &
    (date_accuracy_check['planning'] == False)
].shape[0]

total_count = len(date_accuracy_check)

date_accuracy = (1 - (date_inconsitency_count / total_count)) * 100

print(f"Date Accuracy: {date_accuracy:.2f}%")

Considering confidence score and Human approved as confidence score is been derived from ML model may be wrong is some case and high confidence score must match with Human approval

In [ ]:
high_conf = date_accuracy_check["confidence_score"]>0.8

# if high confidence is true and human approved  is true or high confidence is false or human approved is false that will give us true
agreement = ((high_conf & date_accuracy_check["human_approved"]) | (~high_conf & ~date_accuracy_check["human_approved"]))
aggreement_accuracy = agreement.mean()*100

print(f"Confidence and Human approved: {aggreement_accuracy:.2f}%")


Overall Accuracy percentage

In [ ]:
accuracy = (date_accuracy + aggreement_accuracy)/2
print(f"Overall Accuracy: {accuracy:.2f}%")

## Completeness

Missing or Null value Present in dataframe

In [ ]:
#give overall summary of null and not null value presnt in our data frame
null_column_count = pd.DataFrame({
    'null_value': final_df.isnull().sum(),
    'total_rows':len(final_df),
    'null_percentage': (final_df.isnull().sum() / len(final_df)* 100).round(2)
})

null_column_count= null_column_count.sort_values(by = ['null_percentage','null_value'],ascending = [False,False])

null_column_count

In [ ]:
#Give us overall null and not null percentage

# Data is discrete so null value can't be filled with help of other columns

overall_null_percentage = (null_column_count['null_value'].sum()/
                           null_column_count['total_rows'].sum())*100

print(f"Overall Null Percentage: {overall_null_percentage:.2f}%")

overall_complete = 100 -overall_null_percentage

print(f"Overall Complete: {overall_complete:.2f}%")


## Consistency

Wether value follows same format  across the dataset

In [ ]:
#checking category column format across
final_df.category.value_counts(ascending = False)

Category column is in consistent format

In [ ]:
final_df.planning.value_counts(ascending = False)

Checking if date is consistent

In [ ]:
date_cols = ["found_date","effective_date","published_at"]

date_consistency = {}

for col in date_cols:
    if col in final_df.columns:
      parsed = pd.to_datetime(final_df[col],errors = "coerce",utc=True)
      valid_ratio = parsed.notna().mean()*100
      date_consistency[col] = valid_ratio

print("Date Consistency",date_consistency)

## Timeliness

Data is been updated timely in Database

In [ ]:
check_date_data = final_df[date_cols]

check_date_data.head(20)

In [ ]:
#filtering the data so we will have correct analysis basically remoing the rows where effective date is null
# Selecting the rows where ingestion date > effective date to have better analysis
date_check_data = (
    (final_df["planning"] == False) & (final_df["effective_date"].notna()) &
    (
        pd.to_datetime(final_df["found_date"], utc=True) >= pd.to_datetime(final_df["effective_date"], utc=True)
    )
)

# Convert to datetime only for the calculation (does not overwrite columns)
found_dt = pd.to_datetime(final_df.loc[date_check_data, "found_date"], errors="coerce", utc=True)
eff_dt   = pd.to_datetime(final_df.loc[date_check_data, "effective_date"], errors="coerce", utc=True)

# Calculate lag in days
lag_days = (found_dt - eff_dt).dt.days

# Mean lag
mean_lag = lag_days.mean()
max_lag = lag_days.max()
min_lag = lag_days.min()
median_lag =lag_days.median()


print(f"Average lag (found_date - effective_date): {mean_lag:.2f} days")
print(f"max lag (found_date - effective_date): {max_lag:.2f} days")
print(f"min lag (found_date - effective_date): {min_lag:.2f} days")
print(f"median lag (found_date - effective_date): {median_lag:.2f} days")

In [ ]:
#Plotting the date in box plot to identify the distribution of time across
# Used Plotly Library so that we can have better labels
fig = px.box(lag_days,
             y=lag_days,
             points="outliers",
             title="Lag between Found Date and Effective Date & Planning = False")
fig.update_yaxes(title="Lag (days)")

## Uniqueness

Duplicacy of Data presnt in dataframe

By analysis above we have figured out presnce of duplicate data in few of the column we will,be doing analysis on that selected column only

In [ ]:
id_unique = final_df["id"].nunique()/len(final_df)*100
print(f"Uniquesness of id: {id_unique:.2f}%")

id_article_uniqueness = final_df["id_article"].nunique()/len(final_df)*100
print(f"article_id uniqueness: {id_article_uniqueness:.2f}%")


combo_uniquness = final_df[["summary","found_date"]].drop_duplicates().shape[0]/len(final_df)*100
print(f"combo_uniquness : {combo_uniquness:.2f}%")

overall_uniqueness = (id_unique+id_article_uniqueness+combo_uniquness)/3
print(f"Overall Uniqueness: {overall_uniqueness:.2f}%")

## Validition

If data is correct in correct format. Logic defined in column are correct

In [ ]:
#checking data types of the columns
final_df.dtypes

In [ ]:
final_df[["amount_normalized","headcount","product_fuzzy_match"]].dtypes

Value variation presnt in Dataframe this will give us more consince view which column need to be corrected

In [ ]:
final_df.describe()

In [ ]:
final_df.amount_normalized.value_counts()

As we checked above there were few rows whose found date was before the effective date or published date.
This is also a major error in data

In [ ]:
date_validation = final_df.copy()

date_validation['found_date'] = pd.to_datetime(date_validation['found_date'])
date_validation['effective_date']= pd.to_datetime(date_validation['effective_date'],utc=True)

date_validation["reference_date"] = date_validation["effective_date"].fillna(date_validation["published_at"])

validation = (date_validation["planning"] == False) & (date_validation["effective_date"].notna()& date_validation["reference_date"].notna())

validation_final  = (date_validation.loc[validation,"reference_date"] <= date_validation.loc[validation,"found_date"]).mean()*100
print(f"date validation :{validation_final:.2f}%")

Sample of rows where found_date>effectivce date or publsihed date

In [ ]:
fails = date_validation.loc[
    validation & (date_validation["found_date"] < date_validation["reference_date"]),
    ["id","summary","found_date","effective_date","published_at","reference_date","planning","category","summary","confidence_score","human_approved"]
]
print(f"Size of df is {fails.shape}")
fails.head(10)

In [ ]:
fails.category.value_counts()

## Data Quality

This section basically focus on correcting the data flaws present in data and fixing the issue.

Removing Duplicates

In [ ]:
print(f"Shape of data frame before removing duplicates {final_df.shape}")
final_df = final_df.drop_duplicates()
print(f"Shape of data frame after removing duplicates {final_df.shape}")

As we have checked in Duplicacy part we have find article id is also getting duplicated here we will be figuring out the article id with issue and making correction.

In [ ]:
final_df.id_article.value_counts(ascending=False)

In [ ]:
 duplicate_row_check = final_df[final_df['id_article']=='13d038e6-ecaf-4a2f-8ad7-2fd093f8d090']
 duplicate_row_check.head(6)

To remove the Duplicate article id we have used row number approach here where we will be sorting our data based on effective date desc and id. To pick and filter the data

In [ ]:
final_df = final_df.sort_values(by = ["effective_date","id"],ascending = [False,True])

final_df["row_number"] = final_df.groupby(["id_article"]).cumcount()+1
print(f"shape of dataframe{final_df.shape}")
final_df.head(5)


We only selected the rows which have row number 1 to remove the duplicay in our data

In [ ]:
final_df_filter = final_df[final_df["row_number"]==1]
final_df_filter = final_df_filter.drop("row_number",axis=1)
final_df_filter.shape

Cross Checking with related columns to see the prescene of duplicacy

In [ ]:
counts = final_df_filter[["summary"]].value_counts().reset_index(name="occurrence")
counts.head(10)

In [ ]:
final_df_filter = final_df_filter.sort_values(by = ["effective_date","id"],ascending = [False,True])

final_df_filter["row_number"] = final_df_filter.groupby(["summary"]).cumcount()+1
print(f"shape of dataframe{final_df_filter.shape}")

Removing the Rows where summary  is similar

In [ ]:
final_df_ = final_df_filter[final_df_filter["row_number"]==1]
final_df_ = final_df_.drop("row_number",axis=1)
final_df_.shape

In [ ]:
counts = final_df_[["summary"]].value_counts().reset_index(name="occurrence")
counts.head(10)

In [ ]:
final_df_.id_article.value_counts(ascending=False)

In [ ]:
final_df_.dtypes

As above dtypes says effective date dtype is db date where will be facing issue whicle having comparison with other date type columns

In [ ]:
final_df_['found_date'] = pd.to_datetime(final_df_['found_date'])
final_df_['effective_date']= pd.to_datetime(final_df_['effective_date'],utc=True)

In [ ]:
final_df_[["effective_date","published_at","found_date"]].dtypes

Product fuzzy match column is in string dtype but while anlaysing we found out that it is boolean.Fixing the daata type.
Also Headcount and amount normalized is in float and negative which can't be possible so correcting the datatype and fixing the negativ error

In [ ]:
final_df_["product_fuzzy_match"] = final_df_["product_fuzzy_match"].astype(bool)
final_df_[["amount_normalized","headcount"]] = final_df_[["amount_normalized","headcount"]].astype("Int64")

In [ ]:
final_df_["amount_normalized"] = pd.to_numeric(final_df_["amount_normalized"],errors = 'coerce')
final_df_["headcount"] = pd.to_numeric(final_df_["headcount"],errors = 'coerce')


final_df_["amount_normalized"] = final_df_["amount_normalized"].abs()
final_df_["headcount"] = final_df_["headcount"].abs()

In [ ]:
#Check of data types
final_df_.dtypes

In [ ]:
#distribution of numerical value
final_df_.describe()

In [ ]:
high_conf = date_accuracy_check["confidence_score"]>0.8

# if high confidence is true and human approved  is true or high confidence is false or human approved is false that will give us true
agreement = ((high_conf & date_accuracy_check["human_approved"]) | (~high_conf & ~date_accuracy_check["human_approved"]))
aggreement_accuracy = agreement.mean()*100

print(f"Confidence and Human approved: {aggreement_accuracy:.2f}%")

In [ ]:
id_unique = final_df_["id"].nunique()/len(final_df_)*100
print(f"Uniquesness of id: {id_unique:.2f}%")

id_article_uniqueness = final_df_["id_article"].nunique()/len(final_df_)*100
print(f"article_id uniqueness: {id_article_uniqueness:.2f}%")


combo_uniquness = final_df_[["summary","found_date"]].drop_duplicates().shape[0]/len(final_df_)*100
print(f"combo_uniquness : {combo_uniquness:.2f}%")

overall_uniqueness = (id_unique+id_article_uniqueness+combo_uniquness)/3
print(f"Overall Uniqueness: {overall_uniqueness:.2f}%")

In [ ]:
final_df_.shape